## Aggregate tree level data to subplot scale

This code provides plot level values for AGB and AGB change.
It takes the data from *PeruTrees.csv* and saves results to *PeruSubPlots.csv*

In [102]:
import pandas as pd
import os
from glob import glob
import numpy as np
import geopandas as gpd
from matplotlib import pyplot as plt

plt.style.use(['dark_background'])

In [90]:
WOR_DIR = '/home/s1332488/Fieldwork/Peru'
os.chdir(WOR_DIR)
datafiles = glob('*.csv')
print(datafiles)

['PeruTrees.csv', 'PeruPlots.csv']


In [91]:
# G_trees = pd.read_csv('GabonTrees.csv',index_col='ID')
P_trees = pd.read_csv('PeruTrees.csv',index_col='ID')
# G_plots = pd.read_csv('GabonPlots.csv',index_col='Plot')
P_plots = pd.DataFrame(index=['PC1a','PC2a','PC3a','PC4a'
                             ,'PC1b','PC2b','PC3b','PC4b'
                             ,'PC1c','PC2c','PC3c','PC4c'
                             ,'PC1d','PC2d','PC3d','PC4d'])


In [126]:
def error(df,field,rel_err):
    # df: dataframe with trees
    # field: str, name of field e.g. tlsAGB_19
    # relative error for each value, e.g. 0.28
    # Returns plot level error in that field
    df['square'] = df[field]**2
    sumsq = df.square.sum()
    return rel_err * np.sqrt( sumsq )

def get_subplot_level_data(sp):
    """
    Take tree level database, and aggregate to plot level statistics
    T : Tree level dataframe
    P : Plot level dataframe
    returns | P with all the stats
    """
    result = pd.Series(dtype=np.float64)
    years = [c.split('_')[1] for c in sp.columns if 'comments' in c]
#     PLOTS = T.groupby('plot')
    LOGS = sp[sp.log==1]
#     #-----------------------#
#     for agb in ['ChaveAGB_'+Y for Y in years]:
#         result[agb] = sp.sum()[agb]
#     for vol in ['ChaveVOL_'+Y for Y in years]:
#         result[vol] = sp.sum()[vol]
#     for collateral in ['collateral_'+Y for Y in years[1:]]:
#         P[collateral] = PLOTS.sum()[collateral]
    result['N_trees'] = sp.count().X
    result['N_logged'] = LOGS.count().X
    # 
    notlogs = sp[sp.log!=1]
    logsAGB = LOGS.tlsAGB_19.sum()
    Delta_logsAGB = error(LOGS,'tlsAGB_19',0.28)
    # not logs use dbh and Chave
    notlogs_pre = notlogs.ChaveAGB_19.sum()
    Delta_notlogs_pre = error(notlogs,'ChaveAGB_19',0.5)
    notlogs_post = notlogs.ChaveAGB_20.sum()
    Delta_notlogs_post = error(notlogs,'ChaveAGB_20',0.5)
    change_notLogs = notlogs_post - notlogs_pre
    Delta_changenotLogs = np.abs(change_notLogs*0.25)
    result['AGB_pre'] = logsAGB + notlogs_pre
    result['dAGB_pre'] = np.sqrt(Delta_logsAGB**2 + Delta_notlogs_pre**2)
    result['AGB_post'] = notlogs_post
    result['dAGB_post'] = Delta_notlogs_post
    result['AGB_loss'] = logsAGB + (notlogs_pre - notlogs_post)
    result['dAGB_loss'] = Delta_logsAGB + Delta_changenotLogs
    result['AGB_loss_pct'] = 100* result.AGB_loss / result.AGB_pre
    result['dAGB_loss_pct'] = result.AGB_loss_pct * np.sqrt((result.dAGB_loss / result.AGB_loss)**2 
                                                  + (Delta_notlogs_pre / notlogs_pre)**2)
    return result

In [127]:
def get_subplot_(p):
    plot = P_trees[P_trees['plot']==p]
    xmin = plot.X.min()
    xmax = plot.X.max()
    ymin = plot.Y.min()
    ymax = plot.Y.max()
    
    xcut = (xmax-xmin)/2.0
    ycut = (ymax-ymin)/2.0
    
    sp1 = plot[(plot.X<xcut)*(plot.Y<ycut)]
    sp2 = plot[(plot.X<xcut)*(plot.Y>ycut)]
    sp3 = plot[(plot.X>xcut)*(plot.Y<ycut)]
    sp4 = plot[(plot.X>xcut)*(plot.Y>ycut)]
    
    dataseries=[]
    for sp, i in zip([sp1,sp2,sp3,sp4],['a','b','c','d']):
        ds = get_subplot_level_data(sp).rename(p+i)
        ds['x_coord'] = round(sp.X.mean())
        ds['y_coord'] = round(sp.Y.mean())
        dataseries.append(ds)
    return dataseries


In [128]:
dataframes = []
for pp in ['PC1','PC2','PC3','PC4']:
    dataframes.append(pd.DataFrame(get_subplot_(pp)))

subplots = pd.concat(dataframes)


subplots['geometry'] = 16*['']
geoms = gpd.read_file('/home/s1332488/Chapter3/subplots_peru_geom.shp')
geoms['subplot']=[x[0:2].upper()+x[2:] for x in list(geoms.subplot)]
subplots = pd.merge(subplots,geoms,left_index=True,right_on='subplot')
subplots = subplots.drop('geometry_x',axis=1).rename({'geometry_y':'geometry'},axis=1)
gpd.GeoDataFrame(subplots).to_file('/home/s1332488/Chapter3/subplots_peru.shp')

/home/s1332488/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/s1332488/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  del sys.path[0]


In [119]:
subplots.set_index('subplot')

,N_trees,N_logged,AGB_pre,dAGB_pre,AGB_post,dAGB_post,AGB_loss,dAGB_loss,AGB_loss_pct,dAGB_loss_pct,x_coord,y_coord,geometry
subplot,,,,,,,,,,,,,
PC1a,180.0,2.0,58.844169,5.436712,52.449830,5.151766,6.394338,1.638324,10.866562,2.987194,23.0,21.0,"POLYGON ((-69.72334 -11.03299, -69.72334 -11.0..."
PC1b,153.0,2.0,97.031714,22.180352,54.759937,7.892019,42.271777,1.117034,43.564908,10.588233,24.0,75.0,"POLYGON ((-69.72334 -11.03253, -69.72334 -11.0..."
PC1c,202.0,2.0,51.756284,4.177296,44.235616,4.093303,7.520668,1.356447,14.530927,2.913220,75.0,24.0,"POLYGON ((-69.72283 -11.03255, -69.72240 -11.0..."
PC1d,176.0,1.0,78.069072,7.384231,53.396471,4.784440,24.672602,3.401041,31.603554,5.371412,75.0,75.0,"POLYGON ((-69.72286 -11.03207, -69.72239 -11.0..."
PC2a,233.0,2.0,83.752301,6.039959,78.838541,6.215885,4.913761,1.860633,5.867016,2.266190,30.0,26.0,"POLYGON ((-69.72310 -11.02764, -69.72361 -11.0..."
PC2b,157.0,2.0,81.352875,8.051451,74.786487,8.033859,6.566388,1.026415,8.071489,1.519197,27.0,59.0,"POLYGON ((-69.72338 -11.02694, -69.72315 -11.0..."
PC2c,186.0,0.0,96.174405,14.132601,84.354712,10.710871,11.819693,0.000000,12.289853,1.805965,108.0,26.0,"POLYGON ((-69.72387 -11.02653, -69.72338 -11.0..."
PC2d,90.0,0.0,43.760366,5.426740,43.386717,4.973618,0.373649,0.000000,0.853852,0.105887,112.0,61.0,"POLYGON ((-69.72338 -11.02694, -69.72387 -11.0..."
PC3a,227.0,1.0,130.925093,19.188486,125.606235,20.432694,5.318859,1.069728,4.062520,1.021052,37.0,24.0,"POLYGON ((-69.72240 -11.02397, -69.72216 -11.0..."
